### DA algorithm and scoring for men-proposing

In [48]:
def deferred_acceptance(m_prefs, w_prefs):
    # Initialize all men as free and not yet proposed to anyone
    free_men = list(m_prefs.keys())
    
    # Track the next woman to propose to for each man
    next_proposal_index = {m: 0 for m in m_prefs}  
    
    # Keep track of current matches (woman: man)
    matches = {}  

    while free_men:
        # Take a free man
        man = free_men.pop(0)  
        
        # His next preferred woman
        woman = m_prefs[man][next_proposal_index[man]]  
        
        # Move to next woman for future proposals
        next_proposal_index[man] += 1  

        if woman not in matches:
            # Woman is free, they become matched
            matches[woman] = man
            
        else:
            # Woman is currently matched, check if she prefers this new proposal
            current_match = matches[woman]
            
            if w_prefs[woman].index(man) < w_prefs[woman].index(current_match):
                # She prefers new man
                matches[woman] = man
                
                # Previous man is now free
                free_men.append(current_match)  
            else:
                # She prefers her current match, rejected man remains free
                free_men.append(man)  

    # Converting matches to desired format (man: woman)
    final_matches = {v: k for k, v in matches.items()}
    
    # Sorting keys alphanumerically
    final_matches = dict(sorted(final_matches.items()))
    
    return final_matches


def matching_score(final_matches, m_prefs, w_prefs):
    men_score, women_score = 0, 0

    for man, woman in final_matches.items():
        # Score for men: Find the rank of the woman in this man's preferences
        men_score += m_prefs[man].index(woman) + 1

        # Score for women: Find the rank of the man in this woman's preferences
        women_score += w_prefs[woman].index(man) + 1

    return men_score, women_score, men_score + women_score

### Define the preferences

In [49]:
m_prefs = {
    'm1': ['w2', 'w4', 'w5', 'w1', 'w3'],
    'm2': ['w3', 'w2', 'w4', 'w1', 'w5'],
    'm3': ['w1', 'w5', 'w4', 'w3', 'w2'],
    'm4': ['w4', 'w2', 'w3', 'w1', 'w5'],
    'm5': ['w2', 'w3', 'w5', 'w1', 'w4']
}

w_prefs = {
    'w1': ['m4', 'm2', 'm1', 'm5', 'm3'],
    'w2': ['m2', 'm4', 'm1', 'm5', 'm3'],
    'w3': ['m4', 'm2', 'm1', 'm3', 'm5'],
    'w4': ['m2', 'm1', 'm4', 'm5', 'm3'],
    'w5': ['m1', 'm4', 'm2', 'm3', 'm5']
}

### Running DA for men-proposing

In [50]:
mp_matches = deferred_acceptance(m_prefs, w_prefs)
print(matches)

men_score, women_score, total_score = matching_score(mp_matches, m_prefs, w_prefs)
print("Men's Score:", men_score)
print("Women's Score:", women_score)
print("Total Score:", total_score)

{'w1': 'm5', 'w2': 'm2', 'w3': 'm4', 'w4': 'm1', 'w5': 'm3'}
Men's Score: 7
Women's Score: 18
Total Score: 25


### Running DA for women-proposing

In [51]:
wp_matches = deferred_acceptance(w_prefs, m_prefs)
print(matches)

men_score, women_score, total_score = matching_score(wp_matches, w_prefs, m_prefs)
print("Men's Score:", men_score)
print("Women's Score:", women_score)
print("Total Score:", total_score)

{'w1': 'm5', 'w2': 'm2', 'w3': 'm4', 'w4': 'm1', 'w5': 'm3'}
Men's Score: 12
Women's Score: 13
Total Score: 25


### Check if a matching is stable

In [52]:
def is_stable_matching(final_matches, m_prefs, w_prefs):
    # Invert final_matches for easy access of woman's match
    inverted_matches = {v: k for k, v in final_matches.items()}

    for man, his_match in final_matches.items():
        # Get the list of women preferred over his current match
        preferred_women = m_prefs[man][:m_prefs[man].index(his_match)]

        for woman in preferred_women:
            # Get the man to whom the woman is currently matched
            her_match = inverted_matches[woman]

            # Check if this woman prefers this man over her current match
            if w_prefs[woman].index(man) < w_prefs[woman].index(her_match):
                return False  

    # If no such pair is found, the matching is stable
    return True

### Checking matching stability via examples

In [53]:
m_prefs = {
    'm1': ['w2', 'w4', 'w5', 'w1', 'w3'],
    'm2': ['w3', 'w2', 'w4', 'w1', 'w5'],
    'm3': ['w1', 'w5', 'w4', 'w3', 'w2'],
    'm4': ['w4', 'w2', 'w3', 'w1', 'w5'],
    'm5': ['w2', 'w3', 'w5', 'w1', 'w4']
}

w_prefs = {
    'w1': ['m4', 'm2', 'm1', 'm5', 'm3'],
    'w2': ['m2', 'm4', 'm1', 'm5', 'm3'],
    'w3': ['m4', 'm2', 'm1', 'm3', 'm5'],
    'w4': ['m2', 'm1', 'm4', 'm5', 'm3'],
    'w5': ['m1', 'm4', 'm2', 'm3', 'm5']
}

## these should all be stable
# layer 1
final_matches_1 = {'m1': 'w2', 'm2': 'w3', 'm3': 'w1', 'm4': 'w4', 'm5': 'w5'}

# layer 2
final_matches_2 = {'m1': 'w4', 'm2': 'w3', 'm3': 'w1', 'm4': 'w2', 'm5': 'w5'}
final_matches_3 = {'m1': 'w2', 'm2': 'w3', 'm3': 'w5', 'm4': 'w4', 'm5': 'w1'}

# layer 3
final_matches_4 = {'m1': 'w4', 'm2': 'w2', 'm3': 'w1', 'm4': 'w3', 'm5': 'w5'}
final_matches_5 = {'m1': 'w4', 'm2': 'w3', 'm3': 'w5', 'm4': 'w2', 'm5': 'w1'}

# layer 4
final_matches_6 = {'m1': 'w4', 'm2': 'w3', 'm3': 'w5', 'm4': 'w2', 'm5': 'w1'}

# unstable example
final_matches_7 = {'m1': 'w3', 'm2': 'w4', 'm3': 'w5', 'm4': 'w2', 'm5': 'w1'}

is_stable = is_stable_matching(final_matches_1, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_2, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_3, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_4, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_5, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_6, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

is_stable = is_stable_matching(final_matches_7, m_prefs, w_prefs)
print("The matching is stable" if is_stable else "The matching is unstable")

The matching is stable
The matching is stable
The matching is stable
The matching is stable
The matching is stable
The matching is stable
The matching is unstable


### Lattice construction (naive)

In [ ]:
# generate all matchings, filtering only for stable ones

# start building lattice:

# level 1: mp optimal
# level 2: generate all 1-swap permutations of mp optimal, filtering only for stable ones
# level 3: generate 



def lattice_builder()

In [55]:
def join_matchings(mu, mu_prime, m_prefs):
    # Invert mu_prime for easier comparison
    mu_prime_inverted = {v: k for k, v in mu_prime.items()}

    # Initialize the new matching
    lambda_matching = {}

    for man in mu:
        woman_mu = mu[man]
        woman_mu_prime = mu_prime_inverted[man]

        # Check which woman is preferred by the man
        if m_prefs[man].index(woman_mu) < m_prefs[man].index(woman_mu_prime):
            # He prefers the woman in mu
            lambda_matching[man] = woman_mu
        else:
            # He prefers the woman in mu_prime
            lambda_matching[man] = woman_mu_prime

    return lambda_matching

# Example usage
mu = {'m1': 'w2', 'm2': 'w3', 'm3': 'w1', 'm4': 'w4', 'm5': 'w5'}
mu_prime = {'w1': 'm5', 'w2': 'm2', 'w3': 'm4', 'w4': 'm1', 'w5': 'm3'}

# Assuming m_prefs is defined somewhere in your code
# m_prefs = {...} 

lambda_matching = join_matchings(mu, mu_prime, m_prefs)
print(lambda_matching)


{'m1': 'w2', 'm2': 'w3', 'm3': 'w1', 'm4': 'w4', 'm5': 'w5'}
